In [ ]:
import xarray as xr
from cablab import Cube
import numpy.ma as ma
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import warnings
warnings.filterwarnings('ignore')

In [ ]:
MERCATOR_PROJECTION = dict(projection='merc',llcrnrlat=-60,urcrnrlat=80, llcrnrlon=-180,urcrnrlon=180,lat_ts=20,resolution='c')
ECKERT4_PROJECTION = dict(projection='eck4',lon_0=0,resolution='c')
KAVRAYSKIY7_PROJECTION = dict(projection = "kav7", lon_0=0., resolution=None)

In [ ]:
country_cube_dataset = xr.open_dataset("/home/jovyan/work/datacube/country-cube-0.1.0/country_cube.nc")
country_cube_array = country_cube_dataset.to_array(dim="Indicator_Codes")

In [ ]:
def plot_variable_2d(variable_name, year, title, projection = KAVRAYSKIY7_PROJECTION, save_image = False, **kwargs):
    var_array = country_cube_array.sel(Indicator_Codes = variable_name,Year=year)
    map_plot_ccube(var_array, title, proj = projection, save_me=save_image, **kwargs)

def map_plot_ccube(single_var_array, title_str, **kwargs):
    ESDC = Cube.open("/home/jovyan/work/datacube/cablab-datacube-0.2.3/low-res")
    country_code_map = "/home/jovyan/work/datacube/country-cube-0.1.0/country-codes.csv"
    country_mask = ESDC.data.dataset("country_mask")
    water_mask = ESDC.data.dataset("water_mask")
    lons, lats = np.meshgrid(country_mask.lon,country_mask.lat)
    country_mask_2D = country_mask["country_mask"][0].values
    water_mask_2D = water_mask["water_mask"][0].values
    cdict = read_ESDC_countrycodes(country_code_map)
    idict = {v: k for k, v in cdict.items()}
    
    res, el = expand_to_latlon(single_var_array, country_mask_2D, idict)

    ma_res = ma.array(res, mask =np.isnan(res))
    
    if "vmin" in kwargs:
        vmin = kwargs["vmin"] 
    else:
        vmin = None
    if "vmax" in kwargs:
        vmax = kwargs["vmax"] 
    else:
        vmax = None
    if "proj" in kwargs:
        proj = kwargs["proj"]
    else:
        proj = dict(projection = "kav7", lon_0=0., resolution=None)
        
    fig = plt.figure()
    ax = fig.add_axes([0.05,0.05,0.9,0.9])
    m = Basemap(**proj)
    m.drawmapboundary(fill_color='0.3')
    ccmap = plt.cm.jet
    ccmap.set_bad("gray",1.)
    im = m.pcolormesh(lons,lats,ma_res,shading='flat',cmap=ccmap,latlon=True, vmin = vmin, vmax=vmax)
    #im2 = m.pcolormesh(lons,lats,nodata,shading='flat',cmap=plt.cm.gist_gray,latlon=True)
    m.drawparallels(np.arange(-90.,99.,30.))
    m.drawmeridians(np.arange(-180.,180.,60.))
    cb = m.colorbar(im,"bottom", size="5%", pad="2%")
    ax.set_title(title_str)
    if "save_me" in kwargs:
        if kwargs["save_me"] == True:
            plt.savefig(title_str[0:15] + '.png',dpi = 600)
    ESDC.close()
    return fig, ax, m

def read_ESDC_countrycodes(country_code_map_csv, code_type = 'ISO3166-1-Alpha-3'):
    df = pd.read_csv(country_code_map_csv)
    cc_dict = df.set_index('ISO3166-1-numeric').to_dict()
    return cc_dict[code_type]

def expand_to_latlon(dat_array, cm_2D, idict):
    latlon_array = np.empty(cm_2D.shape)
    latlon_array.fill(np.nan)
    for el in dat_array:
        #print(el.Country_Codes.values, str(el.Country_Codes.values) in idict)
        if str(el.Country_Codes.values) in idict:
            cv = idict[str(el.Country_Codes.values)]
            ii = cm_2D == cv
            #print(ii.shape, latlon_array.shape)
            latlon_array[ii] = el.values
        # el.Country_Codes.values == "ABW" 
    return latlon_array, el

def plot_covariance(variable1_name, variable2_name, title):
    variable1 = country_cube_array.sel(Indicator_Codes = variable1_name)
    variable2 = country_cube_array.sel(Indicator_Codes = variable2_name)
    cov = get_covariance(variable1, variable2)
    map_plot_ccube(cov, title)

def plot_corr_coeff(variable1_name, variable2_name, title):
    variable1 = country_cube_array.sel(Indicator_Codes = variable1_name)
    variable2 = country_cube_array.sel(Indicator_Codes = variable2_name)
    cov = get_covariance(variable1, variable2)
    variable1_std = variable1.std(dim="Year")
    variable2_std = variable2.std(dim="Year")
    corr_coeff = get_corr_coeff(cov, variable1_std, variable2_std)
    map_plot_ccube(corr_coeff, title)
    
def get_covariance(variable1_array, variable2_array):
    X = (variable1_array - variable1_array.mean(dim="Year", skipna=True))
    Y = (variable2_array - variable2_array.mean(dim="Year", skipna=True))
    xxyy = X * Y
    return xxyy.mean(dim="Year")

def get_corr_coeff(cov, std1, std2):
    return cov / (std1 * std2)

In [ ]:
burnt_area_2010 = country_cube_array.sel(Indicator_Codes = "burnt_area",Year=2010)

In [ ]:
plot_variable_2d("evaporation", 2010, "Evaporation, ESDC, 2010", projection = MERCATOR_PROJECTION, save_image=True)

plot_variable_2d("SP.POP.GROW", 2010, "Annual Population Growth [%], 2010, Worldbank", vmin = -2.,vmax = 4.)

plot_variable_2d("NY.GDP.PCAP.CD", 2010, "GDP per capita [$], 2010, Worldbank", projection = ECKERT4_PROJECTION, vmin = 0.,vmax = 50000.)


In [ ]:
burnt_area = country_cube_array.sel(Indicator_Codes = "burnt_area")
pop_growth = country_cube_array.sel(Indicator_Codes = "SP.POP.GROW")
burnt_area_pop_growth_cov = get_covariance(burnt_area, pop_growth)

In [ ]:
plot_covariance("air_temperature_2m", "NY.GDP.PCAP.CD", "Covariance of air temperature and GDP per capita")

In [ ]:
plot_corr_coeff("precipitation", "NV.AGR.TOTL.ZS", "Correlation coefficient of precipitation and Agriculture, value added (% of GDP)")